In [1]:
import sys
import os
import random
import evaluate

import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelWithLMHead, GPTJForQuestionAnswering
from sklearn.model_selection import train_test_split

from datasets import load_dataset, Dataset

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
selection = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(selection)
if selection=="t5-small":
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
elif selection=="MaRiOrOsSi/t5-base-finetuned-question-answering":
    model = AutoModelWithLMHead.from_pretrained("yazdipour/text-to-sparql-t5-base")
else:
    model = GPTJForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-gptj")

/home/jstil/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [3]:
# set seed
random.seed(0)
np.random.seed(0)

In [6]:
dataset = load_dataset("lc_quad")

train = dataset["train"].to_pandas()
train = train.dropna(subset=['question', 'sparql_wikidata', 'NNQT_question']) # replace NNQT_question with wikibase output context & sparql wikidata with answer
train = Dataset.from_pandas(train)
train = train.train_test_split(test_size=0.2)

# test = dataset["test"].to_pandas()
# test = test.dropna(subset=['question', 'sparql_wikidata'])
# test = Dataset.from_pandas(test)

Found cached dataset lc_quad (/home/jstil/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
X = "question"
Y = "sparql_wikidata"
C = "NNQT_question"
prefix = "translate English to SPARQL: "
sacrebleu = evaluate.load("sacrebleu")

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples[X]]
    inputs = tokenizer(
        questions,
        examples[C],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    inputs["answers"] = examples[Y]

    return inputs